# Add Metis Project 2 Data to RDS Database (on AWS Instance)

In [109]:
import psycopg2
import sys
import numpy as np
import pandas as pd
import pandas.io.sql as pdsql
import sqlalchemy as sq

def connect_to_db_sqlalchemy():
    conn_string = "postgresql+psycopg2://"+SENS.dsn_uid+":"+SENS.dsn_pwd+"@"+SENS.dsn_hostname+":"+SENS.dsn_port+"/"+SENS.dsn_database
    engine = sq.create_engine(conn_string)
    return engine

def connect_to_db():
    import SENSITIVE as SENS
    try:
        conn_string = "host="+SENS.dsn_hostname+" port="+SENS.dsn_port+" dbname="+SENS.dsn_database+" user="+SENS.dsn_uid+" password="+SENS.dsn_pwd
        print("Connecting to database\n  ->%s" % (conn_string.replace(SENS.dsn_pwd, '#'*len(SENS.dsn_pwd))))
        conn=psycopg2.connect(conn_string)
        print("Connected!\n")
        return conn
    except:
        print("Unable to connect to the database.")
        return

In [93]:
conn = connect_to_db()
# conn.close()

Connecting to database
  ->host=postgresql-aws.c45uzy6noe0z.us-east-1.rds.amazonaws.com port=5432 dbname=MetisProject2 user=cmgiler password=########
Connected!



In [94]:
# Print available databases:
cursor = conn.cursor()
cursor.execute("""SELECT datname from pg_database""")
rows = cursor.fetchall()
print("\nShow me the databases:\n")
for row in rows:
    print("   ", row[0])


Show me the databases:

    template0
    rdsadmin
    template1
    postgres
    MetisProject2


## IMDb Datasets

Subsets of IMDb data are available for access to customers for personal and non-commercial use. You can hold local copies of this data, and it is subject to our terms and conditions. Please refer to the Non-Commercial Licensing and copyright/license and verify compliance.

### Data Location 

The dataset files can be accessed and downloaded from https://datasets.imdbws.com/. The data is refreshed daily.

### IMDb Dataset Details 

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name. The available datasets are as follows: 

##### title.basics.tsv.gz - Contains the following information for titles:

- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title.
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year.
- endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
- runtimeMinutes – primary runtime of the title, in minutes
- genres (string array) – includes up to three genres associated with the title

In [113]:
conn = connect_to_db()
cursor = conn.cursor()
table_name = 'title_basics'
query = """CREATE TABLE %s(tconst VARCHAR(20) PRIMARY KEY, 
                           titleType VARCHAR(150), 
                           primaryTitle VARCHAR(500), 
                           originalTitle VARCHAR(500), 
                           isAdult BOOLEAN, 
                           startYear INTEGER, 
                           endYear INTEGER, 
                           runtimeMinutes INTEGER, 
                           genre VARCHAR(150))""" % table_name

cursor.execute("DROP TABLE IF EXISTS %s" % table_name)
cursor.execute(query)
conn.commit()
conn.close()

Connecting to database
  ->host=postgresql-aws.c45uzy6noe0z.us-east-1.rds.amazonaws.com port=5432 dbname=MetisProject2 user=cmgiler password=########
Connected!



In [104]:
title_basics_df = pd.read_csv('Data/title.basics.tsv', sep='\t')
def clean_year(y):
    import numpy as np
    try:
        return int(y)
    except:
        return -9999

def clean_genre(y):
    y = str(y)
    if y == '\\N':
        return ''
    return y.split(',')[0].strip()
import datetime
import numpy as np
print(len(title_basics_df))
# title_basics_df.drop('endYear', axis=1, inplace=True)
title_basics_df['endYear'] = title_basics_df['endYear'].apply(clean_year)
title_basics_df['startYear'] = title_basics_df['startYear'].apply(clean_year)
title_basics_df['runtimeMinutes'] = title_basics_df['runtimeMinutes'].apply(clean_year)
title_basics_df['genres'] = title_basics_df['genres'].apply(clean_genre)
title_basics_df['isAdult'] = title_basics_df['isAdult'].apply(bool)
title_basics_df.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


4753215


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,False,1894,-9999,1,Documentary
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,False,1892,-9999,5,Animation
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,False,1892,-9999,4,Animation
3,tt0000004,short,Un bon bock,Un bon bock,False,1892,-9999,-9999,Animation
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,False,1893,-9999,1,Short


In [ ]:
engine = connect_to_db_sqlalchemy()

In [ ]:
title_basics_df.iloc[:100].to_sql(table_name, engine)

In [105]:
def generate_query(table_name, df):
    query = "INSERT INTO " + table_name + " VALUES "
    for i, row in df.iterrows():
        query += str(tuple(row.values))
        query += ', '
    return query[:-2]

In [108]:
s = generate_query(table_name, title_basics_df.iloc[:100])
print(s)

INSERT INTO title_basics VALUES ('tt0000001', 'short', 'Carmencita', 'Carmencita', False, 1894, -9999, 1, 'Documentary'), ('tt0000002', 'short', 'Le clown et ses chiens', 'Le clown et ses chiens', False, 1892, -9999, 5, 'Animation'), ('tt0000003', 'short', 'Pauvre Pierrot', 'Pauvre Pierrot', False, 1892, -9999, 4, 'Animation'), ('tt0000004', 'short', 'Un bon bock', 'Un bon bock', False, 1892, -9999, -9999, 'Animation'), ('tt0000005', 'short', 'Blacksmith Scene', 'Blacksmith Scene', False, 1893, -9999, 1, 'Short'), ('tt0000006', 'short', 'Chinese Opium Den', 'Chinese Opium Den', False, 1894, -9999, 1, 'Short'), ('tt0000007', 'short', 'Corbett and Courtney Before the Kinetograph', 'Corbett and Courtney Before the Kinetograph', False, 1894, -9999, 1, 'Short'), ('tt0000008', 'short', 'Edison Kinetoscopic Record of a Sneeze', 'Edison Kinetoscopic Record of a Sneeze', False, 1894, -9999, 1, 'Documentary'), ('tt0000009', 'movie', 'Miss Jerry', 'Miss Jerry', False, 1894, -9999, 45, 'Romance'),

In [106]:
conn = connect_to_db()
cursor = conn.cursor()
cursor.execute(s)
cursor.commit()
conn.close()

Connecting to database
  ->host=postgresql-aws.c45uzy6noe0z.us-east-1.rds.amazonaws.com port=5432 dbname=MetisProject2 user=cmgiler password=########
Connected!



ProgrammingError: column "La sortie de l'usine Lumière à Lyon" does not exist
LINE 1: ...'short', 'Employees Leaving the Lumière Factory', "La sortie...
                                                             ^


##### title.crew.tsv.gz – Contains the director and writer information for all the titles in IMDb. Fields include:
- tconst (string)
- directors (array of nconsts) - director(s) of the given title
- writers (array of nconsts) – writer(s) of the given title

In [40]:
table_name = 'title_crew'
query = """CREATE TABLE %s(tconst VARCHAR(20) PRIMARY KEY, 
                           directors VARCHAR(500), 
                           writers VARCHAR(500))""" % table_name

cursor.execute("DROP TABLE IF EXISTS %s" % table_name)
cursor.execute(query)
conn.commit()

##### title.episode.tsv.gz – Contains the tv episode information. Fields include:
- tconst (string) - alphanumeric identifier of episode
- parentTconst (string) - alphanumeric identifier of the parent TV Series
- seasonNumber (integer) – season number the episode belongs to
- episodeNumber (integer) – episode number of the tconst in the TV series.

In [41]:
table_name = 'title_episode'
query = """CREATE TABLE %s(tconst VARCHAR(20) PRIMARY KEY, 
                           parentTconst VARCHAR(20), 
                           seasonNumber INTEGER,
                           episodeNumber INTEGER)""" % table_name

cursor.execute("DROP TABLE IF EXISTS %s" % table_name)
cursor.execute(query)
conn.commit()

##### title.principals.tsv.gz – Contains the principal cast/crew for titles
- tconst (string)
- principalCast (array of nconsts) – title’s top-billed cast/crew

In [42]:
table_name = 'title_principals'
query = """CREATE TABLE %s(tconst VARCHAR(20) PRIMARY KEY, 
                           principalCast VARCHAR(500))""" % table_name

cursor.execute("DROP TABLE IF EXISTS %s" % table_name)
cursor.execute(query)
conn.commit()

##### title.ratings.tsv.gz – Contains the IMDb rating and votes information for titles
- tconst (string)
- averageRating – weighted average of all the individual user ratings
- numVotes - number of votes the title has received

In [43]:
table_name = 'title_ratings'
query = """CREATE TABLE %s(tconst VARCHAR(20) PRIMARY KEY, 
                           averageRating REAL,
                           numVotes INTEGER)""" % table_name

cursor.execute("DROP TABLE IF EXISTS %s" % table_name)
cursor.execute(query)
conn.commit()

##### name.basics.tsv.gz – Contains the following information for names:
- nconst (string) - alphanumeric unique identifier of the name/person
- primaryName (string)– name by which the person is most often credited
- birthYear – in YYYY format
- deathYear – in YYYY format if applicable, else ‘\N’
- primaryProfession (array of strings)– the top-3 professions of the person
- knownForTitles (array of tconsts) – titles the person is known for

In [84]:
table_name = 'name_basics'
query = """CREATE TABLE %s(nconst VARCHAR(20) PRIMARY KEY, 
                           prmaryName VARCHAR(250),
                           birthYear INTEGER,
                           deathYear INTEGER,
                           primaryProfession VARCHAR(500),
                           knownForTitles VARCHAR(500))""" % table_name

cursor.execute("DROP TABLE IF EXISTS %s" % table_name)
cursor.execute(query)
conn.commit()